In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("DF_LAB_3").getOrCreate()

Bài 1: Hãy đọc dữ liệu từ các file csv, sử dụng tự suy ra kiểu dữ liệu cho mỗi cột.

In [ ]:
read_csv = lambda path: spark.read.format("csv").options(
    header="true", inferSchema="true", delimiter=";"
).load(path)


orders = read_csv("data/Orders.csv")
customers = read_csv("data/Customer_List.csv")
order_items = read_csv("data/Order_Items.csv")
products = read_csv("data/Products.csv")
order_reviews = read_csv("data/Order_Reviews.csv")

In [ ]:
orders.show(5)

In [ ]:

customers.show(5)

In [ ]:
order_items.show(5)

In [ ]:
products.show(5)

In [ ]:
order_reviews.show(5)

Câu 2: Thống kê tổng số đơn hàng, số lượng khách hàng và người bán. 

In [ ]:
# Tổng số đơn hàng
totalOrders = orders_Df.select("Order_ID").distinct().count()
print("Tổng số đơn hàng:", totalOrders)

# Số lượng khách hàng
totalCustomers = customerList_Df.select("Customer_Trx_ID").distinct().count()
print("Số khách hàng:", totalCustomers)

# Số lượng người bán
totalSellers = orderItems_Df.select("Seller_ID").distinct().count()
print("Số người bán:", totalSellers)

Câu 3: Phân tích số lượng đơn hàng theo quốc gia, sắp xếp theo thứ tự giảm dần. 

In [ ]:
ordersByCountry = orders.join(customers, on='Customer_Trx_ID', how='inner')
orders_per_country = ordersByCountry.groupBy("Customer_Country")\
                                    .count()\
                                    .orderBy("count", ascending = False)

orders_per_country.show(9)

Câu 4: Phân tích số lượng đơn hàng nhóm theo năm, tháng đặt hàng (Hiển thị theo năm 
tăng dần, tháng giảm dần)

In [ ]:
orders_with_time = orders.withColumn(
    "Order_Year", year("Order_Purchase_Timestamp")
).withColumn(
    "Order_Month", month("Order_Purchase_Timestamp")
)

In [ ]:
orders_per_year_month = orders_with_time.groupBy(
    "Order_Year", "Order_Month"
).count()

In [ ]:
orders_per_year_month = orders_per_year_month.orderBy(
    "Order_Year", ascending=True
).orderBy(
    "Order_Month", ascending=False
)

In [ ]:
orders_per_year_month.show(9)

Câu 5: Thống kê điểm đánh giá trung bình, số lượng đánh giá theo từng mức (ví dụ: 1 đến 
5). 

In [ ]:
clean_reviews = order_reviews.withColumn(
    "Review_Score_int",
    col("Review_Score").cast("int")
).filter(
    col("Review_Score_int").between(1, 5)
)

clean_reviews.show(5)

In [ ]:
review_stats = clean_reviews.groupBy("Review_Score_int") \
    .agg(
        count("*").alias("So_Luong_Danh_Gia"),
        avg("Review_Score_int").alias("Diem_Trung_Binh")
    ) \
    .orderBy("Review_Score_int")

review_stats.show()


Bài 10: Xếp hạng các seller dựa trên tổng doanh thu và số lượng đơn hàng bán được.

In [ ]:
orders_items_join = orders.join(
    order_items,
    on="Order_ID",
    how="inner"
)

orders_items_join.show(5)

In [ ]:
orders_items_revenue = orders_items_join.withColumn(
    "Revenue",
    col("Price") + col("Freight_Value")
)

orders_items_revenue.select(
    "Order_ID", "Seller_ID", "Revenue"
).show(5)

In [ ]:
seller_stats = orders_items_revenue.groupBy("Seller_ID") \
    .agg(
        countDistinct("Order_ID").alias("So_Luong_Don_Hang"),
        sum("Revenue").alias("Tong_Doanh_Thu")
    )

In [ ]:
seller_ranking = seller_stats.orderBy(
    col("Tong_Doanh_Thu").desc()
)

seller_ranking.show(9)